In [1]:
import os
import os.path as osp
import time
import math
from datetime import timedelta
from argparse import ArgumentParser

import torch
from torch import cuda
from torch.utils.data import DataLoader
from torch.optim import lr_scheduler
from torch.cuda.amp import autocast
from torch.cuda.amp import GradScaler
from tqdm import tqdm

from east_dataset import EASTDataset
from dataset import SceneTextDataset
from model import EAST

In [14]:
dataset = SceneTextDataset(
    '/workspace/ojt/OCR/dataset/data/medicine',
    split='train',
    image_size=2048,
    crop_size=1024,
    ignore_tags=['masked', 'excluded-region', 'maintable', 'stamp'],
    color_jitter=False
)

In [16]:
dataset2 = SceneTextDataset(
    '/workspace/ojt/OCR/dataset/data/medical',
    split='train',
    image_size=2048,
    crop_size=1024,
    ignore_tags=['masked', 'excluded-region', 'maintable', 'stamp'],
    color_jitter=False
)

In [18]:
dataset2[2]

(array([[[1., 0., 0.],
         [0., 0., 1.],
         [0., 0., 0.],
         ...,
         [0., 1., 0.],
         [1., 1., 0.],
         [1., 0., 1.]],
 
        [[0., 0., 1.],
         [1., 0., 1.],
         [0., 0., 0.],
         ...,
         [0., 1., 1.],
         [1., 0., 1.],
         [0., 1., 0.]],
 
        [[1., 1., 0.],
         [0., 0., 1.],
         [0., 1., 1.],
         ...,
         [1., 0., 0.],
         [1., 1., 0.],
         [1., 0., 1.]],
 
        ...,
 
        [[1., 0., 0.],
         [0., 0., 1.],
         [1., 1., 0.],
         ...,
         [1., 1., 1.],
         [1., 1., 1.],
         [0., 0., 0.]],
 
        [[0., 0., 0.],
         [0., 0., 0.],
         [1., 1., 1.],
         ...,
         [1., 0., 1.],
         [0., 0., 1.],
         [0., 0., 0.]],
 
        [[1., 0., 1.],
         [0., 1., 1.],
         [0., 1., 1.],
         ...,
         [1., 1., 0.],
         [0., 1., 0.],
         [0., 1., 0.]]], dtype=float32),
 array([[[ 1008.88589124,  -811.35907491

In [15]:
dataset[1]

(array([[[0., 0., 1.],
         [1., 0., 0.],
         [1., 1., 0.],
         ...,
         [0., 1., 1.],
         [0., 0., 0.],
         [0., 0., 1.]],
 
        [[1., 1., 1.],
         [0., 1., 0.],
         [1., 0., 1.],
         ...,
         [1., 1., 1.],
         [0., 1., 0.],
         [1., 0., 0.]],
 
        [[0., 1., 0.],
         [0., 0., 1.],
         [0., 0., 0.],
         ...,
         [1., 1., 1.],
         [0., 1., 1.],
         [1., 1., 0.]],
 
        ...,
 
        [[1., 1., 1.],
         [0., 1., 1.],
         [0., 0., 0.],
         ...,
         [1., 1., 0.],
         [1., 1., 1.],
         [1., 0., 0.]],
 
        [[0., 0., 0.],
         [0., 1., 0.],
         [1., 0., 1.],
         ...,
         [1., 1., 0.],
         [0., 0., 0.],
         [0., 1., 0.]],
 
        [[0., 0., 0.],
         [1., 1., 1.],
         [0., 1., 1.],
         ...,
         [1., 1., 1.],
         [1., 1., 0.],
         [0., 0., 0.]]], dtype=float32),
 array([[[  126.65779089, -1349.25293886

In [7]:
dataset = SceneTextDataset(
    '/workspace/ojt/OCR/dataset/data/medicine',
    split='train',
    image_size=2048,
    crop_size=1024,
    ignore_tags=['masked', 'excluded-region', 'maintable', 'stamp'],
    color_jitter=False
)
dataset = EASTDataset(dataset)
num_batches = math.ceil(len(dataset) / 1)

# DataLoader
train_loader = DataLoader(
    dataset,
    batch_size=1,
    shuffle=True
)

In [12]:
for x in train_loader:
    a= x
    break

In [13]:
a

[tensor([[[[0.0000, 0.0000, 0.0000,  ..., 1.0000, 1.0000, 1.0000],
           [0.0000, 1.0000, 1.0000,  ..., 1.0000, 0.0000, 0.0000],
           [0.0000, 1.0000, 0.0000,  ..., 1.0000, 0.0000, 0.0000],
           ...,
           [1.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
           [1.0000, 1.0000, 0.0000,  ..., 1.0000, 1.0000, 0.0000],
           [1.0000, 0.0000, 0.2721,  ..., 0.0000, 0.0000, 0.0000]],
 
          [[0.0000, 0.0000, 0.0000,  ..., 1.0000, 0.0000, 1.0000],
           [1.0000, 1.0000, 0.0000,  ..., 1.0000, 1.0000, 0.0000],
           [0.0000, 1.0000, 0.0000,  ..., 1.0000, 1.0000, 1.0000],
           ...,
           [1.0000, 0.0000, 0.0000,  ..., 1.0000, 1.0000, 0.0000],
           [0.0000, 1.0000, 1.0000,  ..., 1.0000, 1.0000, 1.0000],
           [0.0000, 0.0000, 0.0000,  ..., 1.0000, 1.0000, 1.0000]],
 
          [[0.0000, 1.0000, 0.0000,  ..., 1.0000, 1.0000, 1.0000],
           [0.0000, 1.0000, 0.0000,  ..., 0.0000, 0.0000, 1.0000],
           [0.0000, 0.00